In [ ]:
from utils import utils_models, utils_gn, utils_dgrd, utils_sig
from config.definitions import ROOT_DIR
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.interpolate import interp1d
import importlib
import numpy as np
import pickle
import os
from scipy import stats
import seaborn as sns
importlib.reload(utils_models)
importlib.reload(utils_gn)
importlib.reload(utils_dgrd)
importlib.reload(utils_sig)

In [ ]:
# read training data
train_raw_data = utils_gn.read_data('train_1238.pkl')
train_raw_data.keys()

In [ ]:
# Plot cc discharge voltage for selected cells in the train cells
data_all = utils_gn.read_data('data_all.pkl')                     # read data for all cycles
data_subset = {k: data_all[k] for k in train_raw_data.keys()}     # get all cycles for train cells
ccv = utils_sig.ccv_signature_features(data_dict=data_subset, n=-1, return_ccv=True) # get constant-current voltage at discharge for all cycles (except the last)

In [ ]:
# Save the generated ccv as pkl
utils_gn.dump_data(
    data=ccv,
    path=f"{ROOT_DIR}/data",
    fname="ccv_data.pkl"
)

In [ ]:
#ccv.keys()
# ccv = utils_gn.read_data(
#     fname="ccv_data.pkl",  
#      path=f"{ROOT_DIR}/data"
# ) 

In [ ]:
selected_cells = ['b1c30', 'b2c30', 'b3c27', 'b8c7'] #['b2c30']
#utils_sig.plot_ccv_features(data_dict=ccv, fname="plots/ccv-evolution-graphical-abstract.pdf", sample_cells=selected_cells, option=2)
utils_sig.plot_ccv_features(
    data_dict=ccv,
    fname=f"{ROOT_DIR}/plots/sig-new-ccv-evol-with-arrows.pdf",
    sample_cells=selected_cells,
    option=2
)

In [ ]:
# Create histogram of EOL, capacity and internal resistance curves
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
cmap = plt.get_cmap('gist_heat', len(data_subset))

for i, cell in enumerate(data_subset.keys()):
    
    capacity = data_subset[cell]['summary']['QDischarge']
    ir = data_subset[cell]['summary']['IR']

    ax[1].plot(capacity, 'o', linewidth=1, markersize=1,  c=cmap(i))
    ax[1].set_xlabel('Cycles', fontsize=16)
    ax[1].set_ylabel('Capacity (Ah)', fontsize=16)
    ax[1].set_ylim([0.8, 1.1])

    
    ax[2].plot(ir, 'o', linewidth=1, markersize=1, c=cmap(i))
    ax[2].set_xlabel('Cycles', fontsize=16)
    ax[2].set_ylabel(r'IR ($\Omega$)', fontsize=16)
    ax[2].set_ylim([0.010, 0.025])

   
ax[0].hist(utils_gn.cycle_life(data_dict=data_subset).values, color='brown', ec='black')
ax[0].set_xlabel('EOL of cells', fontsize=16)
ax[0].set_ylabel('Frequency', fontsize=16)

plt.tight_layout()
plt.savefig(fname=f"{ROOT_DIR}/plots/sig-eolhist-cap-ir-explore.pdf", bbox_inches='tight')